# Cryptography (CC4017) -- Week 4

## Exercice 1

Consider the following polynomials modulo 2:

$$ x^3 + x + 1 $$
$$ x^4 + x^2 + 1 $$ 
$$ x^4 + x^3 + x^2 + 1 $$

### Exercicio 1.1
Start with different initial (non-zero) states and test the periods. What can you conclude about the LFSRs?

In [10]:
def lfsr_feedback(register, taps):
    new_bit = 0
    for tap in taps:
        new_bit ^= register[tap]
    return new_bit

def run_lfsr(polynomial, init_state, max_steps=100):
    state = init_state.copy()
    seen_states = {}
    sequence = []
    
    for step in range(max_steps):
        state_tuple = tuple(state)
        if state_tuple in seen_states:
            period = step - seen_states[state_tuple]
            return sequence, period
        
        seen_states[state_tuple] = step
        sequence.append(state[-1])  # Take the output bit
        new_bit = lfsr_feedback(state, polynomial)
        state = [new_bit] + state[:-1]  # Shift register
    
    return sequence, -1

polynomials = {
    "x^3 + x + 1": [2, 0],  # Corresponds to x^3 + x + 1
    "x^4 + x + 1": [3, 0],  # Corresponds to x^4 + x + 1
    "x^4 + x^3 + x^2 + 1": [3, 2, 1]  # Corresponds to x^4 + x^3 + x^2 + 1
}

initial_states = {
    "x^3 + x + 1": [[1, 0, 0], [1, 1, 1], [0, 1, 0]],  # 3-bit states
    "x^4 + x + 1": [[1, 0, 0, 1], [1, 1, 0, 1], [0, 1, 1, 0]],  # 4-bit states
    "x^4 + x^3 + x^2 + 1": [[1, 0, 0, 1], [1, 1, 0, 1], [0, 1, 1, 0]]  # 4-bit states
}

for poly_name, taps in polynomials.items():
    print(f"\nTesting LFSR with polynomial {poly_name}")
    for init_state in initial_states[poly_name]:
        sequence, period = run_lfsr(taps, init_state)
        print(f"Initial state: {init_state} -> Period: {period}, Sequence: {sequence}")


Testing LFSR with polynomial x^3 + x + 1
Initial state: [1, 0, 0] -> Period: 7, Sequence: [0, 0, 1, 1, 1, 0, 1]
Initial state: [1, 1, 1] -> Period: 7, Sequence: [1, 1, 1, 0, 1, 0, 0]
Initial state: [0, 1, 0] -> Period: 7, Sequence: [0, 1, 0, 0, 1, 1, 1]

Testing LFSR with polynomial x^4 + x + 1
Initial state: [1, 0, 0, 1] -> Period: 15, Sequence: [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Initial state: [1, 1, 0, 1] -> Period: 15, Sequence: [1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0]
Initial state: [0, 1, 1, 0] -> Period: 15, Sequence: [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]

Testing LFSR with polynomial x^4 + x^3 + x^2 + 1
Initial state: [1, 0, 0, 1] -> Period: 7, Sequence: [1, 0, 0, 1, 1, 1, 0]
Initial state: [1, 1, 0, 1] -> Period: 7, Sequence: [1, 0, 1, 1, 0, 0, 0]
Initial state: [0, 1, 1, 0] -> Period: 7, Sequence: [0, 1, 1, 0, 0, 0, 1]


The primary conclusion one can draw from testing the periods of these LFSRs is the significance of irreducibility in polynomial selection for maximal-length sequences. Irreducible polynomials ensure that the generated sequences are not only long but also exhibit properties akin to randomness, which are pivotal in cryptographic applications.

### Exercicio 1.2
Can you ascertain which is the best polynomial for an LFSR?

Determining the best polynomial for an LFSR involves examining both irreducibility and the maximum period. <br>

Both $x⁴ + x + 1 $ and $x⁴ + x³ + x² + 1$ can present a maximum period of 15, ($ 2^n -1$), yet one must consider additional factors such as implementation efficiency and ease of state recovery. <br>

Therefore, the polynomial $x⁴ + x + 1 $ could be deemed superior due to its simplicity and well-established cryptographic utility.

### Exercicio 1.3
Check if any of these is an irreducible polynomial in sage. What does this say about the polynomial, when used
in LFSRs?

![alt text](image.png)

If a polynomial is irreducible, it cannot be factored further and is often a good choice for LFSRs. This property ensures that the LFSR will produce a maximum-length sequence if the initial state is non-zero. <br>

If any of the polynomials are **not irreducible**, it could mean they won't generate the maximum period, and thus they may not be as effective for generating pseudo-random sequences.

The last polynomial, $x⁴ + x³ + x² + 1$, is not irreducible, so instead of getting the maximum period of 15, the actual period is 7 to prove that is not effective for getting PRS.

## Exercice 2

Obtain a Python implementation of RC4 from the web and use it to encrypt a file.

In [7]:
from Crypto.Cipher import ARC4 # type: ignore
from Crypto.Random import get_random_bytes # type: ignore
import binascii

# Generate key
key = get_random_bytes(16)  # 128-bit key

# Example plaintext
plaintext = b'Hello, OpenSSL compatibility test!'  # Byte array input

# Create RC4 cipher object
cipher = ARC4.new(key)

# Encrypt the plaintext
ciphertext = cipher.encrypt(plaintext)

# Print key and ciphertext in hexadecimal
print("Key (hex):", binascii.hexlify(key).decode())
print("Ciphertext (hex):", binascii.hexlify(ciphertext).decode())

Key (hex): 96fc06c579f8c350a82e36b477636700
Ciphertext (hex): 14467e981ef2f968d7d78adc31476e61bc23291155266793664de14e1dd7fe96338d


## Exercice 3

Check that this algorithm is compatible with OpenSSL

Using the output from the last exercice:

Key (hex): 96fc06c579f8c350a82e36b477636700 <br>

Ciphertext (hex): 14467e981ef2f968d7d78adc31476e61bc23291155266793664de14e1dd7fe96338d


And the command 

> echo -n "ciphertext" | xxd -r -p | openssl rc4 -d -K key


![alt text](image-1.png)

We can see that the algorithm is compatible with OpenSSL, but we need to pass the values in hexadecimal format.

## Exercice 4

Demonstrate with OpenSSL that ChaCha20 produces a repeated ciphertext if you encrypt the same file with the same key and nonce.

Let's create a simple file to encrypt.

> echo "This is a test file for ChaCha20 encryption." > testfile.txt

Now lets run the openssl with the following command:

> openssl enc -chacha20 -in testfile.txt -out encrypted1.bin -K 00112233445566778899aabbccddeeff00112233445566778899aabbccddeeff -iv 00112233445566


Now lets encrypt the same file again using the same key and nonce

> openssl enc -chacha20 -in testfile.txt -out encrypted2.bin -K 00112233445566778899aabbccddeeff00112233445566778899aabbccddeeff -iv 00112233445566


We can now check if the two ciphertexts are identical using the cmp command

> cmp encrypted1.bin encrypted2.bin

cmp shows no output, meaning that the files are identical, confirming that ChaCha20 produces the same ciphertext when encrypting the same plaintext with the same key and nonce.

## Exercice 5

In questions 2 and 4, compare the size of the plaintext with the size of the ciphertext. What can you conclude
with respect, for example, to AES-CTR and AES-CBC modes studied last week.

ChaCha20 and AES-CTR will produce ciphertexts the same size as the plaintext because they are stream ciphers.

AES-CBC will typically produce a larger ciphertext due to padding

ChaCha20 and AES-CTR are stream ciphers, and their ciphertext size matches the plaintext size.
AES-CBC is a block cipher mode that requires padding, so the ciphertext is typically larger than the plaintext when padding is added.